# Campus Crime and Disciplinary Data

In [63]:
%matplotlib inline

import os
import xlrd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [64]:
ocd09 = xlrd.open_workbook('crime-sheets/Crime2009EXCEL/oncampusdiscipline060708.xls', logfile=open(os.devnull, 'w'))
ocd08 = xlrd.open_workbook('crime-sheets/Crime2008EXCEL/oncampusdiscipline050607.xls', logfile=open(os.devnull, 'w'))
ocd09df = pd.read_excel(ocd09, engine='xlrd')
ocd08df = pd.read_excel(ocd08, engine='xlrd')

In [66]:
ocd16df = pd.read_excel('crime-sheets/Crime2016EXCEL/oncampusdiscipline131415.xls')
ocd15df = pd.read_excel('crime-sheets/Crime2015EXCEL/oncampusdiscipline121314.xls')
ocd14df = pd.read_excel('crime-sheets/Crime2014EXCEL/oncampusdiscipline111213.xls')
ocd13df = pd.read_excel('crime-sheets/Crime2013EXCEL/oncampusdiscipline101112.xls')
ocd12df = pd.read_excel('crime-sheets/Crime2012EXCEL/oncampusdiscipline091011.xls')
ocd11df = pd.read_excel('crime-sheets/Crime2011EXCEL/oncampusdiscipline080910.xls')
ocd10df = pd.read_excel('crime-sheets/Crime2010EXCEL/oncampusdiscipline070809.xls')
#ocd09df = pd.read_excel('crime-sheets/Crime2009EXCEL/oncampusdiscipline060708.xls')
#ocd08df = pd.read_excel('crime-sheets/Crime2008EXCEL/oncampusdiscipline050607.xls')

# rename total, Zip column in 08 for consistency with the rest of the sheets
ocd08df = ocd08df.rename(index=str, columns={'total': 'Total', 'Zip': 'ZIP'}) 
# rename the 09 columsn in 10 report due to typo
ocd10df = ocd10df.rename(index=str, columns={'WEAPON8.1': 'WEAPON9', 'DRUG8.1': 'DRUG9', 'LIQUOR8.1': 'LIQUOR9'}) 
# rename 08 and 09 columns to match all caps format for subsequent years
ocd08df = ocd08df.rename(index=str, columns={'Weapon5': 'WEAPON5', 'Drug5': 'DRUG5', 'Liquor5': 'LIQUOR5',
                                             'Weapon6': 'WEAPON6', 'Drug6': 'DRUG6', 'Liquor6': 'LIQUOR6',
                                             'Weapon7': 'WEAPON7', 'Drug7': 'DRUG7', 'Liquor7': 'LIQUOR7'})
ocd09df = ocd09df.rename(index=str, columns={'Weapon6': 'WEAPON6', 'Drug6': 'DRUG6', 'Liquor6': 'LIQUOR6',
                                             'Weapon7': 'WEAPON7', 'Drug7': 'DRUG7', 'Liquor7': 'LIQUOR7',
                                             'Weapon8': 'WEAPON8', 'Drug8': 'DRUG8', 'Liquor8': 'LIQUOR8'})

ocd_dfs = [ocd16df ,ocd15df, ocd14df, ocd13df, ocd12df, ocd11df, ocd10df, ocd09df, ocd08df]
len(ocd_dfs)

9

In [67]:
# includes the missing zero in the UNIT ID that is present in subsequent years for 08 and 09 data
for i in ocd_dfs[len(ocd_dfs) - 2:]:
  last_digits = map(lambda x: x % 10, i['UNITID_P'])
  #ocd08df['UNITID_P'] = ocd08df.UNITID_P.astype(int)
  i['UNITID_P'] = ((i['UNITID_P'] / 100)).astype(int)
  i['UNITID_P'] = (i['UNITID_P'] * 1000)
  i['UNITID_P'] = map(lambda x, y: x + y, i['UNITID_P'], last_digits)

In [58]:
i = ocd10df
i = i.drop(list(i)[1:12], 1)
i = i[list(i)[:len(list(i))-3]]
list(i)

[u'UNITID_P',
 u'WEAPON7',
 u'DRUG7',
 u'LIQUOR7',
 u'WEAPON8',
 u'DRUG8',
 u'LIQUOR8',
 'WEAPON9',
 'DRUG9',
 'LIQUOR9']

In [60]:
i = ocd08df
i = i.drop(list(i)[1:12], 1)
i = i[list(i)[:len(list(i))-3]]
print list(i)

[u'UNITID_P', u'Weapon5', u'Drug5', u'Liquor5', u'Weapon6', u'Drug6', u'Liquor6', u'Weapon7', u'Drug7', u'Liquor7']


In [68]:
df = pd.DataFrame(ocd16df)
# old cold used for merging intersection of columns
#    on=list(set(list(df)) & set(list(i))

#i = ocd09df
#i = i.drop(list(i)[1:12], 1)
#i = i[list(i)[:len(list(i))-3]]
#df = df.merge(i, how='inner', on=list(set(list(df)) & set(list(i))))

for i in ocd_dfs: 
    i = i.drop(list(i)[1:12], 1)
    i = i[list(i)[:len(list(i))-3]]
    #print list(set(list(df)) & set(list(i)))
    df = df.merge(i, how='inner', on=list(set(list(df)) & set(list(i))))
    df.dropna()
    print df.shape

print 'done'

(11306, 24)
(10620, 27)
(9820, 30)
(9187, 33)
(8661, 36)
(7957, 39)
(7426, 42)
(6829, 45)
(7021, 48)
done


In [75]:
UC_df = df[df['INSTNM'].str.match('University of California*', case=False)]

In [76]:
UC_df = UC_df.drop(['UNITID_P', 'Address', 'BRANCH', 'Address', 'City', 'State', 'ZIP', 'sector_cd', 'Sector_desc', 'FILTER13', 'FILTER14', 'FILTER15', 'men_total', 'women_total', 'Total'], 1)

In [77]:
UC_df = UC_df.groupby(UC_df.INSTNM).sum()

,WEAPON13,DRUG13,LIQUOR13,WEAPON14,DRUG14,LIQUOR14,WEAPON15,DRUG15,LIQUOR15,WEAPON12,...,LIQUOR8,WEAPON7,DRUG7,LIQUOR7,WEAPON6,DRUG6,LIQUOR6,WEAPON5,DRUG5,LIQUOR5
count,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,...,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000
mean,1.111111,147.777778,161.666667,0.777778,47.222222,88.555556,1.777778,129.888889,181.777778,2.111111,...,291.777778,1.666667,121.333333,330.222222,1.666667,107.666667,304.888889,2.777778,84.222222,239.333333
std,1.763834,400.825017,338.430569,1.394433,101.198787,152.987835,4.323707,340.723291,374.957590,3.655285,...,719.527410,2.958040,283.383574,673.805569,2.397916,258.632945,561.730239,5.911383,198.830819,451.138283
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,2.000000,7.000000,0.000000,1.000000,4.000000,0.000000,0.000000,5.000000,0.000000,...,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1.000000,25.000000,41.000000,1.000000,28.000000,70.000000,0.000000,37.000000,82.000000,3.000000,...,56.000000,2.000000,37.000000,109.000000,3.000000,38.000000,133.000000,3.000000,42.000000,101.000000
max,5.000000,1214.000000,992.000000,4.000000,306.000000,386.000000,13.000000,1035.000000,1118.000000,11.000000,...,2182.000000,9.000000,860.000000,1943.000000,6.000000,788.000000,1526.000000,18.000000,605.000000,1257.000000


In [74]:
UC_df.plot.bar(x='INSTNM', y='LIQUOR13')
UC_df.plot.bar(x='INSTNM', y='LIQUOR14')
UC_df.plot.bar(x='INSTNM', y='LIQUOR15')

KeyError: 'INSTNM'